# Deep Q-Learning Agent

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dqn import QAgent

In [ ]:
# define environment

In [ ]:
# define parameters

In [ ]:
# define and train agent

In [ ]:
plt.style.use('ggplot')
ma_window = 100
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), tight_layout=True)

# Plot 1
ax1.plot(returns, label='raw return')
ax1.plot(pd.Series(returns).rolling(ma_window).mean(), label=f'moving average window={ma_window}', color='blue')
ax1.set_title('Returns Over Episodes', fontsize='xx-large')
ax1.set_xlabel('Episode')
ax1.legend()
ax1.set_ylabel('Return')